In [1]:
import os
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style = "ticks")

%matplotlib inline


cwd = os.getcwd()
print("Current working directory: {0}".format(cwd))

realestate = pd.read_csv("./data/Ames Real Estate Data.csv", low_memory = False)
HousePrice = pd.read_csv("./data/Ames_HousePrice.csv")

Current working directory: /Users/chereowang27/NYCDSA/Machine_Learning_I/Machine_Learning_Project


In [6]:
realestate = pd.DataFrame(realestate)
HousePrice = pd.DataFrame(HousePrice)

In [18]:
cols = HousePrice.columns
num_cols = HousePrice._get_numeric_data().columns



In [64]:
categorical = list(set(cols) - set(num_cols))


,SaleType,MiscFeature,RoofMatl,GarageType,FireplaceQu,MSZoning,ExterCond,Fence,RoofStyle,BsmtExposure,...,Functional,Utilities,BsmtFinType1,BsmtFinType2,HouseStyle,Alley,Heating,Exterior1st,GarageQual,Condition1
0,WD,NaN,CompShg,Detchd,Gd,RL,TA,NaN,Gable,No,...,Typ,AllPub,Rec,Unf,1Story,NaN,GasA,Wd Sdng,TA,Norm
1,WD,NaN,CompShg,Attchd,NaN,RL,TA,NaN,Gable,Mn,...,Typ,AllPub,GLQ,ALQ,1Story,NaN,GasA,HdBoard,TA,Norm
2,WD,NaN,CompShg,Detchd,NaN,C (all),TA,NaN,Hip,No,...,Typ,AllPub,ALQ,Unf,1Story,NaN,GasA,MetalSd,TA,Norm
3,WD,NaN,CompShg,Detchd,NaN,RL,Gd,NaN,Gable,No,...,Typ,AllPub,Unf,Unf,2Story,NaN,GasA,MetalSd,TA,Norm
4,WD,NaN,CompShg,Attchd,NaN,RL,TA,NaN,Gable,No,...,Typ,AllPub,GLQ,Unf,2Story,NaN,GasA,VinylSd,TA,Norm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2575,WD,NaN,CompShg,Detchd,Gd,RL,TA,NaN,Gable,No,...,Typ,AllPub,Unf,Unf,1.5Unf,NaN,Grav,Wd Sdng,Fa,Norm
2576,WD,NaN,CompShg,Attchd,Gd,RL,TA,NaN,Hip,NaN,...,Min2,AllPub,NaN,NaN,1Story,NaN,GasA,BrkFace,TA,Norm
2577,WD,NaN,CompShg,2Types,NaN,RH,TA,NaN,Gable,No,...,Typ,AllPub,BLQ,Unf,2Story,NaN,GasA,MetalSd,TA,Norm
2578,WD,NaN,CompShg,Attchd,TA,RL,TA,NaN,Gable,No,...,Typ,AllPub,GLQ,Unf,2Story,NaN,GasA,VinylSd,TA,Norm


In [11]:
for feature in categorical: 
    print(HousePrice[feature].value_counts())
    print('')

TA    2312
Fa     110
Gd      22
Po       4
Ex       3
Name: GarageQual, dtype: int64

MnPrv    301
GdPrv    111
GdWo     103
MnWw      10
Name: Fence, dtype: int64

GLQ    753
Unf    706
ALQ    397
Rec    265
BLQ    251
LwQ    139
Name: BsmtFinType1, dtype: int64

Normal     2414
Partial      82
Abnorml      61
Family       17
Alloca        4
AdjLand       2
Name: SaleCondition, dtype: int64

Reg    1641
IR1     855
IR2      71
IR3      13
Name: LotShape, dtype: int64

Y    2336
N     184
P      60
Name: PavedDrive, dtype: int64

Norm      2228
Feedr      137
Artery      81
RRAn        40
PosN        37
RRAe        26
PosA        19
RRNn         8
RRNe         4
Name: Condition1, dtype: int64

Inside     1886
Corner      439
CulDSac     164
FR2          78
FR3          13
Name: LotConfig, dtype: int64

Gtl    2458
Mod     109
Sev      13
Name: LandSlope, dtype: int64

Y    2417
N     163
Name: CentralAir, dtype: int64

SBrkr    2365
FuseA     168
FuseF      39
FuseP       7
Name: Elec

In [56]:
missingRows = HousePrice.isnull().any(axis = 1)
missingCols = HousePrice.isnull().any(axis = 0)
HousePrice.columns[missingCols]


Index(['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Electrical',
       'BsmtFullBath', 'BsmtHalfBath', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PoolQC', 'Fence', 'MiscFeature'],
      dtype='object')

In [63]:
num_mis = HousePrice._get_numeric_data().isnull().any(axis = 1)
HousePrice._get_numeric_data().isna().sum().sort_values(ascending = False)

LotFrontage      462
GarageYrBlt      129
MasVnrArea        14
BsmtHalfBath       2
BsmtFullBath       2
BsmtUnfSF          1
TotalBsmtSF        1
GarageCars         1
BsmtFinSF1         1
BsmtFinSF2         1
GarageArea         1
TotRmsAbvGrd       0
MoSold             0
Fireplaces         0
MiscVal            0
PoolArea           0
ScreenPorch        0
BedroomAbvGr       0
WoodDeckSF         0
OpenPorchSF        0
EnclosedPorch      0
3SsnPorch          0
KitchenAbvGr       0
Unnamed: 0         0
HalfBath           0
FullBath           0
PID                0
LowQualFinSF       0
2ndFlrSF           0
1stFlrSF           0
YearRemodAdd       0
YearBuilt          0
OverallCond        0
OverallQual        0
LotArea            0
MSSubClass         0
SalePrice          0
GrLivArea          0
YrSold             0
dtype: int64

In [66]:
HousePrice[categorical].isna().sum().sort_values(ascending = False)

PoolQC           2571
MiscFeature      2483
Alley            2412
Fence            2055
FireplaceQu      1241
GarageQual        129
GarageCond        129
GarageFinish      129
GarageType        127
BsmtExposure       71
BsmtFinType2       70
BsmtQual           69
BsmtCond           69
BsmtFinType1       69
MasVnrType         14
Electrical          1
SaleCondition       0
LandSlope           0
Street              0
SaleType            0
Functional          0
Utilities           0
ExterQual           0
HouseStyle          0
Heating             0
Exterior1st         0
BldgType            0
LotConfig           0
KitchenQual         0
Exterior2nd         0
Condition2          0
PavedDrive          0
LotShape            0
LandContour         0
HeatingQC           0
CentralAir          0
Neighborhood        0
Foundation          0
RoofStyle           0
ExterCond           0
MSZoning            0
RoofMatl            0
Condition1          0
dtype: int64

In [43]:
HousePrice[HousePrice.columns[missingCols]].isna().sum().sort_values(ascending = False)

PoolQC          2571
MiscFeature     2483
Alley           2412
Fence           2055
FireplaceQu     1241
LotFrontage      462
GarageCond       129
GarageQual       129
GarageFinish     129
GarageYrBlt      129
GarageType       127
BsmtExposure      71
BsmtFinType2      70
BsmtFinType1      69
BsmtQual          69
BsmtCond          69
MasVnrArea        14
MasVnrType        14
BsmtHalfBath       2
BsmtFullBath       2
TotalBsmtSF        1
BsmtUnfSF          1
GarageCars         1
GarageArea         1
BsmtFinSF2         1
BsmtFinSF1         1
Electrical         1
dtype: int64

ImportError: attempted relative import with no known parent package

In [28]:
(map(HousePrice,is.na)))

SyntaxError: invalid syntax (<ipython-input-28-06ecb4cfafab>, line 1)